# Obtaining Vectors for Test Set

In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
import cv2

Using Theano backend.


In [2]:
base_model = load_model('./VGG/base_model.h5')
top_model = load_model('./VGG/top_model.h5')

/Users/Sam/anaconda2/lib/python2.7/site-packages/keras/models.py:136: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Vectorizing Adoptable Dog Image

In [13]:
import os

In [14]:
#First remove all the erroneous pictures
#Rename all train files to .png to be compatible with VGG Model

# ADOPT_PATH = "./AdoptionPictures/Everything/"
# for filename in os.listdir(ADOPT_PATH):
#     adopt_filepath = ADOPT_PATH + "/" + filename
#     valid_filepath = ADOPT_PATH + "/" + filename + '.png'
#     os.rename(adopt_filepath, valid_filepath)

In [15]:
ADOPT_PATH = "./SamplePictures/"

In [16]:
activation_vectors = []

In [17]:
images = []
for idx, filename in enumerate(os.listdir(ADOPT_PATH)):
    if idx > 20:
        break
    img = cv2.resize(cv2.imread(ADOPT_PATH + str(idx+1) + '.png'), (150, 150)).astype(np.float32)

    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    img = img.transpose((2,0,1))
    images.append(img)
    
images = np.array(images)
bottleneck_features_test = base_model.predict(images)

train_data = np.load(open('./VGG/bottleneck_features_train.npy'))
test_model = Sequential()
test_model.add(Flatten(input_shape=train_data.shape[1:]))
test_model.add(Dense(256, activation='sigmoid', weights=top_model.layers[1].get_weights()))

activation_vectors = test_model.predict(bottleneck_features_test)
np.save("sample_array", activation_vectors)

# Nearest Neighbors

In [3]:
adoptable_array = np.load("adoptable_array.npy")

In [4]:
sample_array = np.load("sample_array.npy")

In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
neigh = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
neigh.fit(adoptable_array)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [7]:
adoptable_cos = []
for dog_ind in range(len(sample_array)): 
    dist_cos, ind_cos = neigh.kneighbors(sample_array[dog_ind], n_neighbors = 6)
    adoptable_cos.extend(ind_cos)

/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.

In [8]:
import json

In [9]:
dog_vectors = [list(dog_vec) for dog_vec in adoptable_cos]

In [19]:
with open("dog_recommendations.json", "wb") as outfile:
    json.dump(dog_vectors, outfile)

In [49]:
recs = [
    [1, 2, 9],
    [3, 5, 6],
    [4, 7, 8],
    [0,1,2]
]

In [50]:
ranked_recs = zip(*recs)

In [51]:
ranked_recs

[(1, 3, 4, 0), (2, 5, 7, 1), (9, 6, 8, 2)]

In [52]:
flattened_recs = []

In [53]:
for items in ranked_recs:
    for item in items:
        temp = item + 1
        flattened_recs.append(temp)

In [54]:
flattened_recs

[2, 4, 5, 1, 3, 6, 8, 2, 10, 7, 9, 3]

In [55]:
recommend_images = []
for pic in flattened_recs:
    recommend_images.append(str(pic) + '.png')

In [56]:
print(recommend_images)

['2.png', '4.png', '5.png', '1.png', '3.png', '6.png', '8.png', '2.png', '10.png', '7.png', '9.png', '3.png']


In [47]:
rec_images_2d = []
rec_row = []
for idx, image in enumerate(recommend_images):
    if idx % 6 == 0 and idx > 0:
        rec_images_2d.append(rec_row)
        rec_row = []
    elif idx == len(recommend_images)-1:
        rec_row.append(image)
        rec_images_2d.append(rec_row)
        rec_row = []
    else:
        print idx
        rec_row.append(image)

0
1
2
3
4
5
7


# Model Function for App

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
import cv2
import os
    

Using TensorFlow backend.


In [5]:

base_model = load_model('./VGG/Theano_base_model.h5')
top_model = load_model('./VGG/Theano_top_model.h5')

train_data = np.load(open('./VGG/bottleneck_features_train.npy'))
test_model = Sequential()
test_model.add(Flatten(input_shape=train_data.shape[1:]))
test_model.add(Dense(256, activation='sigmoid', weights=top_model.layers[1].get_weights()))

adoptable_array = np.load("adoptable_array.npy")
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
neigh.fit(adoptable_array)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [6]:
def recommend_pets(filename):
    
   
    activation_vectors = []
    ADOPT_PATH = "./AdoptionPictures/Everything/"
    images = []
    
    img = cv2.resize(cv2.imread(ADOPT_PATH + filename), (150, 150)).astype(np.float32)
    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    img = img.transpose((2,0,1))
    images.append(img)
    
    
    images = np.array(images)
    print images.shape
    bottleneck_features_test = base_model.predict(images)
    
    activation_vectors = test_model.predict(bottleneck_features_test)
       
    
    adoptable_cos = []
    dist_cos, ind_cos = neigh.kneighbors(activation_vectors, n_neighbors = 6)
    adoptable_cos.extend(ind_cos)
    
    import json
    dog_vectors = [list(dog_vec) for dog_vec in adoptable_cos]
    
    recs = []
    
    for num in dog_vectors[0]:
        recs.append(int(num) + 1)
    

    recommend_images = []
    for pic in recs:
        recommend_images.append(str(pic) + '.png')
    
    return recommend_images
    

In [4]:
recommend_pets('3.png')

(1, 3, 150, 150)


['3.png', '1662.png', '479.png', '1517.png', '1231.png', '464.png']

In [7]:
recommend_pets('3.png')

(1, 3, 150, 150)


['24.png', '2999.png', '460.png', '2238.png', '1057.png', '1732.png']